# IsoAdata  
> This module centers around the **IsoAdata** class, an extension of the AnnData object, specifically designed to handle isoform-resolution single-cell data. It provides specialized methods to efficiently manage and analyze data from various protocols that generate such data, facilitating seamless integration and manipulation within the Allos framework.  

As the landscape of single-cell RNA sequencing evolves, IsoAdata offers a robust and adaptable structure that can be extended to meet specific research needs. While it serves as a comprehensive foundation, we encourage the community to contribute by enhancing its capabilities and developing additional functionalities tailored to unique analytical requirements. This collaborative effort aims to build a dynamic ecosystem where users can leverage shared innovations and protocol-specific enhancements.  

The primary objective is to empower researchers to work effortlessly with isoform-level single-cell RNA sequencing data in Allos, ensuring compatibility and efficiency across diverse datasets while benefiting from community-driven advancements.


In [ ]:
#| default_exp anndata_iso

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
import anndata as ad

In [ ]:
from allos.readers_tests import *
from allos.preprocessing import subset_common_cells, transfer_obs

In [ ]:
combined_mouse_data = process_mouse_data()

Starting download of test data from https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz
File downloaded successfully
File extracted successfully
Test data downloaded successfully
Starting download of test data from https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748089/suppl/GSM3748089%5F951c.isoforms.matrix.txt.gz
File downloaded successfully
File extracted successfully
Test data downloaded successfully


/data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
combined_mouse_data.obs.index

Index(['CCACCTACAAAGTCAA', 'AACCATGAGACCACGA', 'GATGAGGTCGACCAGC',
       'GCACTCTTCTATCGCC', 'TGCCCTAAGGCCCGTT', 'TTCGGTCAGCTGGAAC',
       'GGATTACAGACGCAAC', 'GTCTCGTTCCTCTAGC', 'CCATGTCAGTAGATGT',
       'GATCGATGTCAAACTC',
       ...
       'GCTCCTACATCGACGC', 'AGCGTCGCATTATCTC', 'ATCCACCTCCCACTTG',
       'GTGCTTCCAAGGCTCC', 'GACTACATCATTGCGA', 'TACTCATTCGCCATAA',
       'CGTGAGCTCTCTAAGG', 'TTATGCTAGTAAGTAC', 'CGGACACTCTTGTCAT',
       'GGCTCGAGTACCGAGA'],
      dtype='object', length=1109)

In [ ]:
# | export
class AnnDataIso(ad.AnnData):
    # Private method to filter dataset for genes with multiple isoforms.
    def _filter_isodata(self):
        """
        Filters the data to include only genes with multiple isoforms.

        Logic:
        - Counts the number of isoforms for each gene using `np.unique`.
        - Creates a DataFrame with gene IDs and their isoform counts.
        - Identifies genes with more than one isoform.
        - Subsets the AnnData object to include only these genes.

        Returns:
        - Filtered AnnData object with only genes having multiple isoforms.
        """
        genes, frequencies = np.unique(self.var['geneId'], return_counts=True)
        df = pd.DataFrame({"geneId": genes, "frequencies": frequencies})
        multi_iso_genes = df[df["frequencies"] > 1]['geneId'].tolist()
        filtr_adata_iso = self[:, self.var['geneId'].isin(multi_iso_genes)]
        return filtr_adata_iso

    # Method to compute isoform expression percentages relative to total gene expression.
    def iso_percent(self, df, barcodes_regex="^[ACGT]+$"):
        """
        Calculates isoform expression percentages for each barcode.

        Parameters:
        - df (DataFrame): Input DataFrame containing isoform expression data.
        - barcodes_regex (str): Regex to identify columns representing barcodes.

        Returns:
        - DataFrame with normalized isoform percentages for each barcode.
        """
        iso_perc_df = df.__deepcopy__()
        if len(iso_perc_df.filter(regex=(barcodes_regex)).columns.to_list()) < 1:
            raise ValueError("No barcode was identified. Please check the names of the columns.")
        # Normalize isoform counts by gene-level expression.
        iso_perc_df[iso_perc_df.filter(regex=(barcodes_regex)).columns.to_list()] = (
            iso_perc_df.filter(regex=(barcodes_regex)) /
            iso_perc_df.groupby(['geneId']).transform('sum').filter(regex=(barcodes_regex))
        )
        # Replace NaN values with 0.
        iso_perc_df = iso_perc_df.replace(np.nan, 0.0)
        return iso_perc_df

    # Class initializer to process the AnnData object and compute relevant annotations.
    def __init__(self, anndata: ad.AnnData, palette='ghibli'):
        """
        Initializes the AnnDataIso object with isoform and gene-level annotations.

        Parameters:
        - anndata (AnnData): Input AnnData object containing isoform-level data.
        - cell_types (DataFrame): DataFrame with cell type annotations.
        """
        self._init_as_actual(anndata.copy())
        # Group genes and count the number of isoforms.
        self.gene_counts = self.var.reset_index().groupby(by='geneId').count()
        # Filter dataset for genes with multiple isoforms.
        self._filtered_anndata = self._filter_isodata()
        # Annotate cell types in the observation data.
        # Compute isoform percentages.
        df = self._filtered_anndata.to_df().set_index(self._filtered_anndata.obs['barcodes'])
        df = df.transpose()
        df[['transcriptId', 'geneId']] = self._filtered_anndata.var
        df_m_iso = self.iso_percent(df)
        df_m_iso = df_m_iso.iloc[0:, :-2].transpose()
        self._filtered_anndata.obsm['Iso_prct'] = df_m_iso
        self.colors = ['#6c6246', '#387041', '#153c27', '#393525', '#b19c77', '#175934', '#d0e7f3', '#466175', '#8c8c5a', '#47462c', '#8c414e', '#579f4e', '#7cb282', '#30363a', '#672c37', '#e5c6ad', '#402b2f', '#b0606e', '#c1e5d8', '#829ea7']
        if palette != 'ghibli':
            self.colors = ['#100856', '#7243ee', '#da2ad3', '#3a58a4', '#2d149e', '#7f0d9a', '#3b0554', '#7a97f4', '#a127b4', '#696e43', '#de62e2', '#4e1af5', '#88904e', '#2d2c18', '#576d5c', '#4a4f20', '#909cfa', '#929574', '#c905ca', '#71a493']


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()